<a href="https://colab.research.google.com/github/elooo3/Masters-NLP---B620035/blob/main/Logistic_Regression_ISEAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dependencies

In [ ]:
# importing libraries to be used for performing set tasks
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re # for simplifying the reviews
import nltk # for downloading ensemble of stop words
nltk.download('wordnet')
nltk.download('stopwords') # now, download stop words
from nltk.corpus import stopwords # import stop words into notebook
from nltk.stem.porter import PorterStemmer # import class to be used in performing stemming 
from nltk.stem import WordNetLemmatizer # import class to be used in performing lematization

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\obemb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\obemb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# importing dataset
dataset = pd.read_excel('Dataset.xlsx', sheet_name = 'isear')

In [ ]:
from nltk.corpus import stopwords

# creating an argument that holds all stop words in english language
stop_words = stopwords.words('english')

# Text Preprocessing

In [ ]:
corpus = [] # create a list which will contain all cleaned data
for i in range(0, 7511): # where 7511 is the number of rows in the dataset 
#  store data in review and update after every cleaning process
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i]) # re subfunction to replace any element that is not a letter with a space
  review = review.lower() # transform all capitals to lowercase letters 
  review = review.split() # split the different elements of the sentences into different words preparing it for stemming
  ps = PorterStemmer() # call stemming function
  lemma = WordNetLemmatizer() #call lemmatizer function
  all_stopwords = stopwords.words('english') # defining stop words in english
 
  # defining a list of words to be removed from stop word list
  unwanted_num = {'not','is','but','why','before','again','how','more','most','no','don','will','wouldn','against','aren','couldn','didn','doesn','hadn','hasn','haven','isn','wasn','weren'}
 
  all_stopwords = [ele for ele in all_stopwords if ele not in unwanted_num] # remove all words specified above from stop word list

  review = [lemma.lemmatize(word) for word in review if not word in set(all_stopwords)] # lemmatize words in the sentences for each review
  review = ' '.join(review) # get back original format of the sentence 
  corpus.append(review) # update corpus with each clean sentences

# Feature Engineering/Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer #import tfidf vectorizer for tokenization

# create instance of the tfidf vectorizer class
tfid = TfidfVectorizer(smooth_idf=False)

X = tfid.fit_transform(corpus).toarray() # fit corpus to X
 
y = dataset.loc[:, ['Label']].values # set target variable as the emotional states

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0) #split dataset into training and test sets in ratio 75:25

# Model Fitting

In [ ]:
# importing necessary libraries
import pandas as pd 
from sklearn.linear_model  import Ridge,Lasso,RidgeCV, LassoCV, ElasticNet, ElasticNetCV, LogisticRegression 
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skl
sns.set()

In [ ]:
#fit model on dataset logistic regressor
lr = LogisticRegression()
lr.fit(X_train,y_train)

y_hat = lr.predict(X_train)


accuracy_score(y_train, y_hat) # give accuracy score on training set

C:\Users\obemb\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.8455529913012604

In [ ]:
# test model on unseen data
y_pred = lr.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.62      0.76      0.68       250
           2       0.71      0.73      0.72       280
           3       0.46      0.45      0.45       272
           4       0.70      0.57      0.63       287
           5       0.58      0.64      0.61       280
           6       0.48      0.44      0.46       256
           7       0.50      0.50      0.50       253

    accuracy                           0.58      1878
   macro avg       0.58      0.58      0.58      1878
weighted avg       0.58      0.58      0.58      1878



# Hyperparameter tuning

In [ ]:
lr = lr = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs')
lr.fit(X_train,y_train)
y_hat = lr.predict(X_train)
accuracy_score(y_train, y_hat)

C:\Users\obemb\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.8455529913012604

In [ ]:
# let's fit the data into kNN model and see how well it performs:
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs', class_weight = 'balanced', penalty = 'l2')
param_grid = { 
               'C' : [0.8,1,1.5]
              }
# param_grid = { 
 #              'C' : [0.2,0.4,0.6,0.8,1,1.5]
  #            }     
gridsearch = GridSearchCV(lr, param_grid,verbose=3, n_jobs = 3)
gridsearch.fit(X_train,y_train)
# let's see the  best parameters according to gridsearch
#gridsearch.best_params_
gridsearch.best_score_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


C:\Users\obemb\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\obemb\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5751803385652663

In [ ]:
lr = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs', class_weight = 'balanced', penalty = 'l2', C = 1.5)
lr.fit(X_train,y_train)
y_hat = lr.predict(X_train)
accuracy_score(y_train, y_hat)

C:\Users\obemb\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\obemb\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8812355760695899

In [ ]:
y_pred = lr.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.64      0.74      0.69       250
           2       0.71      0.72      0.71       280
           3       0.46      0.44      0.45       272
           4       0.69      0.59      0.63       287
           5       0.59      0.65      0.62       280
           6       0.47      0.44      0.46       256
           7       0.49      0.49      0.49       253

    accuracy                           0.58      1878
   macro avg       0.58      0.58      0.58      1878
weighted avg       0.58      0.58      0.58      1878



In [ ]:
y_prob_pred = lr.predict_proba(X_test)
roc_auc_score(y_test, y_prob_pred, multi_class='ovr', average='weighted')

0.8624099652644618

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train,y_hat))

              precision    recall  f1-score   support

           1       0.85      0.92      0.89       838
           2       0.87      0.90      0.88       803
           3       0.78      0.82      0.80       812
           4       0.88      0.83      0.86       793
           5       0.85      0.85      0.85       787
           6       0.87      0.80      0.83       793
           7       0.82      0.78      0.80       807

    accuracy                           0.85      5633
   macro avg       0.85      0.85      0.85      5633
weighted avg       0.85      0.85      0.85      5633

